# DO ZROBIENIA

- wszystko ma być w funkcjach
- tłumaczenie na angielski wszystkich plików i odwołań do nich tutaj
- przerobienie całego "constants" na słownik config = {} i wszystkie odwołania do stałych przerzucić na takie
- eleganckie po kolei wuywołanie wszystkiego

- trzymamy się jednej konwencji nazewniczej do funkcji... niech to będzie już `gen_cośtam()`

- tłumaczenie id of purchase w tabelach

- rentale nie mogą być oddane w nocy

1. wszystkie stałe, liczby itp do słownika `config` i odwołania w funkcjach
2. funkcje maja być bezargumentowe, oprócz trych które będą udekorowane jako @staticmethod itp. ale to jest zaznaczone
4. jest wywołanie tych funkcji wszystkich w jednej komórce

```python
sales = jakiś dataframe
sales = ogranij_cos_z_sales()

```

In [1]:
import datetime
import os
import random
from itertools import repeat
from pathlib import Path
import itertools
import holidays
import matplotlib.pyplot as plt
import numpy.typing as npt
import numpy as np
import pandas as pd
from faker import Faker
from scipy.stats import expon, gamma, norm, randint, skewnorm, uniform
from unidecode import unidecode_expect_ascii

C:\Users\kjani\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = {
    'staff_number': 6,

    'pay_day': 5,

    'customers_number': 1500,

    'shop_lifetime_years': 2,

    'traffic_decrease_magnitude': 4,

    'shop_open_hours': {"from": 8, "to": 20, "pick": 17},

    'traffic_std': 1.5,

    'rental_to_sales_ratio': 0.25,

    'salary_settings': {"minimal": 3530, "exp_scale": 100},

    'weekday_dict': {
        0: "Monday",
        1: "Tuesday",
        2: "Wednesday",
        3: "Thursday",
        4: "Friday",
        5: "Saturday",
    },

    'customer_pcs_probas': {
        1: 0.7,
        2: 0.25,
        3: 0.05,
    },
    
    'staff_shifts': {"morning": [1, 2], "afternoon": [3, 4, 6], "fired": [5]},

    'event_info': {"weekday": 4, "hour": 15},

    'initial_customer_n': 25,

    'daily_customer_increment': 0.015,

    'weekly_extras_multip': 1,

    'weekly_extras': {
        0: 2,
        1: 1,
        2: 0,
        3: 4,
        4: 5,
        5: 5,
    },

    'bulk_ratio': 0.35,

    'inventory_multiplier': 1, 

    'phone_num_prefixes': [45, 50, 51, 53, 57, 60, 66, 69, 72, 73, 78, 79, 88],

    'avg_supply_yearly_rate': 3,

    'rental_games_n': 70,

    'inactive_rental_games': 3,

    'rental_price_ratio': 0.2,

    'rental_allowed_days': 3,
    
    'penalty_ratio': 0.3
}

In [3]:

# DODATKOWE ATRYBUTY POTRZEBNE DO LOGIKI !!!!!!!!!!
phone_base = []
email_base = []


### Uwaga, musi być baza telefonów i emaili, bo między customerami i staff też muszą być one różne...

## Wczytanie csv

In [4]:
def file_path(file_name: str):
    dir_path = [i for i in Path.cwd().parents if i.name == "geeks-and-dragons-db"][0]
    path = os.path.join(dir_path, "config", "prompts", file_name)
    if os.path.exists(path):
        return path
    else:
        raise FileNotFoundError("No such file")

In [5]:
prompt_games = pd.read_csv(file_path("games.csv"))
prompt_first_names_males = pd.read_csv(file_path("names_males.csv"))
prompt_last_names_males = pd.read_csv(file_path("lastnames_males.csv"))
prompt_first_names_females = pd.read_csv(file_path("names_females.csv"))
prompt_last_names_females = pd.read_csv(file_path("lastnames_females.csv"))
prompt_cities = pd.read_csv(file_path("cities.csv"))
prompt_emails = pd.read_csv(file_path("domains.csv"))
prompt_tournaments = pd.read_csv(file_path("tournaments.csv"))

________
# OSTATECZNA WERSJA

In [6]:
def gen_prompt_dates():
    # all the dates
    prompt_dates = pd.DataFrame(
        {
            "date": pd.date_range(
                end=datetime.datetime.now().date(), periods=365 * config['shop_lifetime_years']
            )
        }
    )
    # weekdays
    prompt_dates["weekday"] = prompt_dates["date"].apply(lambda date: date.weekday())
    prompt_dates["weekday_name"] = prompt_dates["weekday"].map(config['weekday_dict'])
    # filtering sundays and holidays
    holidays_pl = holidays.Poland()
    holiday_mask = prompt_dates["date"].apply(lambda date: date in holidays_pl)
    prompt_dates = prompt_dates[(prompt_dates["weekday"] != 6) & ~holiday_mask]
    # adding the base traffic volume
    ### initial numbe
    prompt_dates["volume_base"] = config['initial_customer_n']
    ### a linear trend
    prompt_dates["volume_base"] += (
        np.repeat(config['daily_customer_increment'], prompt_dates.shape[0]).cumsum().round()
    )
    ### additional traffic for each day
    prompt_dates["volume_base"] += (
        prompt_dates["weekday"].map(config['weekly_extras']) * config['weekly_extras_multip']
    )
    # adding sales and rental traffic
    ### sales = base + noise
    prompt_dates["volume_sales"] = (
        prompt_dates["volume_base"]
        + np.random.normal(loc=0, scale=config['traffic_std'], size=prompt_dates.shape[0])
    ).round()
    ### rental = base * ratio + noise
    prompt_dates["volume_rental"] = (
        prompt_dates["volume_base"] * config['rental_to_sales_ratio']
        + np.random.normal(loc=0, scale=config['traffic_std'], size=prompt_dates.shape[0])
    ).round()
    prompt_dates = prompt_dates.reset_index(drop=True)
    return prompt_dates


def gen_prompt_hours():
    # list of the hours when opened
    prompt_hours = pd.DataFrame(
        {"hour": np.arange(start=config['shop_open_hours']["from"], stop=config['shop_open_hours']["to"])}
    )
    # traffic increments
    prompt_hours["customer_n"] = np.random.exponential(
        size=prompt_hours.shape[0]
    ).cumsum()
    # traffic decrements after the pick
    decrease_stage = prompt_hours[prompt_hours["hour"] > config['shop_open_hours']["pick"]]
    prompt_hours.loc[decrease_stage.index, "customer_n"] -= np.random.exponential(
        scale=config['traffic_decrease_magnitude'], size=decrease_stage.shape[0]
    ).cumsum()
    # probability column
    prompt_hours.loc[prompt_hours["customer_n"] < 0, "customer_n"] = 0
    prompt_hours["prob"] = prompt_hours["customer_n"] / prompt_hours["customer_n"].sum()
    return prompt_hours


def prepar_prompt_games():
    # sample the games but leave the first in csv as the first there
    prompt_games[1:] = prompt_games[1:].sample(frac=1)
    # add the weights
    rng = np.linspace(0, 2, prompt_games["name"].shape[0])
    expon_curve = expon.pdf(x=rng)
    prompt_games["weights"] = expon_curve / expon_curve.sum()


# @staticmethod - leave the argument as it is!!!!
@np.vectorize
def _gen_one_phone(*args) -> str:
    while True:
        # get the prefix and the rest of digits
        digits = [
            np.random.choice(config['phone_num_prefixes']),
            *np.random.randint(0, 10, size=7),
        ]
        # concat the digits
        phone = "".join(map(str, digits))
        if not phone in phone_base:
            break
    phone_base.append(phone)
    return phone


# @staticmethod - leave the argument as it is!!!!
@np.vectorize
def _gen_one_name(gender: str, mode: str) -> str:
    # first or last name colname mode
    if mode == "first":
        col = "first_name"
        dfs = {"M": prompt_first_names_males, "F": prompt_first_names_females}
    elif mode == "last":
        col = "last_name"
        dfs = {"M": prompt_last_names_males, "F": prompt_last_names_females}
    else:
        raise ValueError(f"Cannot generate a name in {mode} mode")
    # random name based on gender and stats
    df = dfs[gender]
    return np.random.choice(df[col], p=df["prob"])


# @staticmethod - leave the argument as it is!!!!
@np.vectorize
def _gen_email(first_name: str, last_name: str) -> str:
    # join the names and cleanse it
    raw_name = f"{first_name.lower()}.{last_name.lower()}"
    cleansed_name = unidecode_expect_ascii(raw_name)
    # initially do not add the number after the name
    number = ""
    while True:
        # concat all the elements
        domain = np.random.choice(prompt_emails["domain"])
        email = f"{cleansed_name}{number}@{domain}"
        # in the second iteration add the number
        number = np.random.randint(0, 10)
        if not email in email_base:
            break
    email_base.append(email)
    return email


# @staticmethod - leave the argument as it is!!!!
def _gen_staff_to_date() -> npt.NDArray:
    # NULL fill
    dates = np.zeros(config['staff_number'], dtype=np.object_)
    dates[:] = np.nan
    # find the constants
    some_employee_ix = -2
    shop_open_date = prompt_dates["date"].iloc[0]
    today = prompt_dates["date"].iloc[-1]
    # fire him
    dates[some_employee_ix] = Faker().date_between(
        shop_open_date + datetime.timedelta(days=90),
        today - datetime.timedelta(days=90),
    )
    return dates


# @staticmethod - leave the argument as it is!!!!
def _gen_staff_form_date(staff: pd.DataFrame) -> npt.NDArray:
    # hire all of them
    dates = np.zeros(config['staff_number'], dtype=np.object_)
    dates[:] = prompt_dates["date"].iloc[0]
    # hire one with a time shift
    some_employee_ix = -1
    ones_end_date = staff["to_date"].loc[staff["to_date"].isnull() == False].values[0]
    dates[some_employee_ix] = ones_end_date + datetime.timedelta(days=30)
    return dates


def _gen_staff_update_time(staff: pd.DataFrame) -> npt.NDArray:
    df_date = staff[["to_date", "from_date"]]
    return df_date.max(axis=1, skipna=True, numeric_only=False).apply(
        lambda date: date
        + pd.DateOffset(
            hours=random.choice(range(8, 20)),
            minutes=random.choice(range(0, 60)),
            seconds=random.choice(range(0, 60)),
        )
    ) 


# @staticmethod - leave the argument as it is!!!!
def _gen_salary(staff: pd.DataFrame) -> npt.NDArray:
    # over the base salary
    salaries = np.random.exponential(
        scale=config['salary_settings']["exp_scale"],
        size=staff.shape[0],
    )
    # add the base
    salaries += config['salary_settings']["minimal"]
    # round it
    salaries = salaries.round(2)
    # remove the salary for the fired ones
    ix = staff.loc[staff["to_date"].isnull() == False]
    salaries[ix.index] = np.nan
    return salaries


# @staticmethod - leave the argument as it is!!!!
def _gen_manager_status(staff: pd.DataFrame) -> npt.NDArray:
    # on entry no managers
    status = np.full(staff.shape[0], False)
    # the one who has the greatest salary becoms a manager
    ix = staff.loc[staff["current_salary"] == staff["current_salary"].max()]
    status[ix.index] = True
    return status


def gen_staff():
    # basic info
    staff = pd.DataFrame(
        {
            "gender": np.random.choice(["M", "F"], size=config['staff_number']),
            "phone": _gen_one_phone(np.zeros(config['staff_number'])),
            "to_date": _gen_staff_to_date(),
        }
    )
    # details
    staff["city"] = np.full(staff.shape[0], "Wrocław")
    staff["from_date"] = _gen_staff_form_date(staff)
    staff["first_name"] = _gen_one_name(staff["gender"], "first")
    staff["last_name"] = _gen_one_name(staff["gender"], "last")
    staff["email"] = _gen_email(staff["first_name"], staff["last_name"])
    staff["current_salary"] = _gen_salary(staff)
    staff["is_manager"] = _gen_manager_status(staff)
    staff["updated_at"] = _gen_staff_update_time(staff)
    # indexing
    staff.sort_values(by=["updated_at"], inplace=True)
    staff = staff.reset_index(drop=True)
    staff["staff_id"] = staff.reset_index()["index"] + 1
    staff = staff.reindex(
        labels=[
            "staff_id",
            "first_name",
            "last_name",
            "phone",
            "email",
            "city",
            "current_salary",
            "is_manager",
            "gender",
            "from_date",
            "to_date",
            "updated_at",
        ],
        axis=1,
    )
    return staff


def gen_prompt_staff_shifts():
    # all the hour-day combinations
    hours = pd.DataFrame(
        {"hour": np.arange(config['shop_open_hours']["from"], config['shop_open_hours']["to"])}
    )
    weekdays = pd.DataFrame({"weekday": np.arange(0, 7)})
    shifts = pd.merge(hours, weekdays, how="cross")
    # get the morning and afternoon shift hours
    morning_rows = shifts[
        ((shifts["weekday"] <= 4) & (shifts["hour"] >= 8) & (shifts["hour"] < 15))
        | ((shifts["weekday"] > 4) & (shifts["hour"] >= 8) & (shifts["hour"] < 13))
    ]
    afternoon_rows = shifts[
        ((shifts["weekday"] <= 3) & (shifts["hour"] >= 14) & (shifts["hour"] < 20))
        | ((shifts["weekday"] > 3) & (shifts["hour"] >= 12) & (shifts["hour"] < 20))
    ]
    # add the employees to each field
    afternoons = pd.merge(
        pd.DataFrame({"staff_id": config['staff_shifts']["afternoon"]}),
        afternoon_rows,
        how="cross",
    )
    mornings = pd.merge(
        pd.DataFrame({"staff_id": config['staff_shifts']["morning"]}), morning_rows, how="cross"
    )
    return pd.concat([mornings, afternoons]).reset_index(drop=True)


# norm rvs with minimum value condition
def norm_with_condition_generator(n, loc, scale, condition):
    dates = norm.rvs(scale=scale, loc=loc, size=n)
    mask = dates > condition
    while mask.all() == False:
        dates = norm.rvs(scale=scale, loc=loc, size=n)
        mask = dates > condition
    return dates.round().astype(int)


def gen_realtionships():
    global staff
    n = int(np.ceil(config['staff_number'] * 1.5))
    staff_id = []
    staff_gender = []
    update = []
    current_staff = staff.loc[staff["current_salary"].isnull() == False]
    for i in range(n):
        employee = random.choices(
            current_staff.staff_id.values, weights=current_staff.current_salary.values
        )
        staff_id.append(*employee)
        staff_gender.append(
            *current_staff["gender"].loc[current_staff.staff_id == employee[0]].values
        )
        update.append(
            Faker().date_between_dates(
                pd.to_datetime(
                    current_staff.from_date.loc[current_staff.staff_id == employee[0]].values[0]
                ),
                prompt_dates["date"].iloc[-1],
            )
            + pd.DateOffset(
                hours=random.choice(range(8, 20)),
                minutes=random.choice(range(0, 60)),
                seconds=random.choice(range(0, 60)),
            )
        )
    df = pd.DataFrame(
        {
            "staff_id": staff_id,
            "staff_gender": staff_gender,
            "dates_number": norm_with_condition_generator(n, 5, 2, 0),
            "updated_at": update,
        }
    )
    df.sort_values(by=["updated_at"], inplace=True)
    df = df.reset_index(drop=True)
    df["relationship_id"] = df.reset_index()['index'] + 1
    df["partner_id"] = df.reset_index()['index'] + 1
    df = df.reindex(
        labels=[
            "relationship_id",
            "staff_id",
            "staff_gender",
            "partner_id",
            "dates_number",
            "updated_at",
        ],
        axis=1,
    )
    return df


def gen_partners():
    global relationships
    partner = relationships.partner_id
    staff_gender = relationships.staff_gender.values
    gender = []
    for el in staff_gender:
        if el == "M":
            gender.append(random.choices(["M", "F"], weights=[0.1, 0.9])[0])
        elif el == "F":
            gender.append(random.choices(["M", "F"], weights=[0.9, 0.1])[0])

    partners = pd.DataFrame(
        {
            "partner_id": partner,
            "gender": gender,
            "updated_at": relationships.updated_at,
        }
    )
    partners["name"] = _gen_one_name(partners["gender"], mode="first")
    partners.sort_values(by=["updated_at"], inplace=True)
    partners = partners.reindex(
        labels=["partner_id", "name", "gender", "updated_at"], axis=1
    )
    relationships.drop(columns=["staff_gender"], inplace=True)
    return partners.reset_index(drop=True)


def _gen_expenses_dates():
    base = pd.DataFrame(
        prompt_dates.loc[
            (prompt_dates["date"].dt.day == config['pay_day'])
            | (prompt_dates["date"].dt.day == config['pay_day'] + 1),
            "date",
        ]
    )
    base["flag"] = base["date"].dt.to_period("M")
    base.drop_duplicates(["flag"], keep="first", inplace=True)
    base = base.reset_index()
    return base["date"]


def _gen_rent_expenses():
    date_df = prompt_dates.loc[prompt_dates["date"].dt.day == 5, "date"]
    title = date_df.dt.month_name(locale="pl_PL").apply(
        lambda x: "CZYNSZ " + str(x).upper()
    )
    df = pd.DataFrame(
        {
            "date": date_df,
            "title": title,
            "amount": np.full((1, date_df.shape[0]), 3250)[0],
            "type": np.full((1, date_df.shape[0]), "CZYNSZ")[0],
        }
    )
    return df


def _gen_energy_expenses():
    date_df = prompt_dates.loc[prompt_dates["date"].dt.day == 5, "date"]
    title = date_df.dt.month_name(locale="pl_PL").apply(
        lambda x: "ENERGIA ELEKTRYCZNA " + str(x).upper()
    )
    df = pd.DataFrame(
        {
            "date": date_df,
            "title": title,
            "amount": [
                max(round(norm.rvs(150, 4), 2), 100) for i in range(date_df.shape[0])
            ],
            "type": np.full((1, date_df.shape[0]), "MEDIA")[0],
        }
    )
    return df


def _gen_water_expenses():
    date_df = prompt_dates.loc[prompt_dates["date"].dt.day == 5, "date"]
    title = date_df.dt.month_name(locale="pl_PL").apply(
        lambda x: "WODA " + str(x).upper()
    )
    df = pd.DataFrame(
        {
            "date": date_df,
            "title": title,
            "amount": [
                max(round(norm.rvs(50, 2), 2), 15) for i in range(date_df.shape[0])
            ],
            "type": np.full((1, date_df.shape[0]), "MEDIA")[0],
        }
    )
    return df


def _gen_heat_expenses():
    dates = []
    date_df = prompt_dates.loc[prompt_dates["date"].dt.day == 5, "date"]
    for j in range(2021, 2024):
        for i in range(4, 11):
            day = datetime.datetime(j, i, 5)
            dates.append(day.strftime("%Y-%m-%d"))
    for i in dates:
        date_df = date_df.drop(date_df[date_df == i].index)
    title = date_df.dt.month_name(locale="pl_PL").apply(
        lambda x: "OGRZEWANIE " + str(x).upper()
    )
    df = pd.DataFrame(
        {
            "date": date_df,
            "title": title,
            "amount": [round(norm.rvs(100, 4), 2) for i in range(date_df.shape[0])],
            "type": np.full((1, date_df.shape[0]), "MEDIA")[0],
        }
    )
    return df


def _gen_salary_expenses():
    date_df = _gen_expenses_dates()
    all_staff = pd.concat(
        [staff[["first_name", "last_name", "current_salary"]]] * date_df.shape[0],
        ignore_index=True,
    )
    staff_name = staff["first_name"].str.cat(staff["last_name"], sep=" ")
    names = pd.concat([staff_name] * date_df.shape[0], ignore_index=True)
    dates = np.repeat(date_df, staff_name.shape[0])
    dates = dates.reset_index()["date"]
    title_first = dates.dt.month_name(locale="pl_PL").apply(
        lambda x: "PENSJA " + str(x).upper()
    )
    title = title_first.str.cat(names, sep=" ")
    df = pd.DataFrame(
        {
            "date": dates,
            "title": title,
            "amount": all_staff["current_salary"],
            "type": np.full((1, title.shape[0]), "PENSJA")[0],
        }
    )
    to_date = pd.to_datetime(
        staff["to_date"].loc[staff["to_date"].isnull() == False].values[0]
    )
    df.loc[(df["amount"].isnull() == True) & (df["date"] < to_date), "amount"] = round(
        3490 + expon.rvs(scale=100, loc=80), 2
    )
    df.dropna(inplace=True)
    return df


def gen_maintenance_expenses():
    df = pd.DataFrame(
        {"date": [], "title": [], "amount": [], "type": [], "updated_at": []}
    )
    df = pd.concat(
        [
            df,
            _gen_rent_expenses(),
            _gen_energy_expenses(),
            _gen_water_expenses(),
            _gen_heat_expenses(),
            _gen_salary_expenses(),
        ]
    )
    df["updated_at"] = df["date"].apply(
        lambda x: x
        + pd.DateOffset(
            hours=random.choice(range(8, 20)),
            minutes=random.choice(range(0, 60)),
            seconds=random.choice(range(0, 60)),
        )
    )
    df.sort_values(by=["updated_at"], inplace=True)
    df = df.reset_index(drop=True)
    df["payment_id"] = df.reset_index()['index'] + 1
    df["invoice_id"] = df.reset_index()['index'] + 1
    df["spend_id"] = df.reset_index()['index'] + 1
    df = df.reindex(
        [
            "spend_id",
            "title",
            "amount",
            "type",
            "date",
            "payment_id",
            "invoice_id",
            "updated_at",
        ],
        axis=1,
    )
    return df


def gen_expense_types():
    df = pd.DataFrame({
        "expenses_type": maintenance_expenses["type"],
        "updated_at": maintenance_expenses["updated_at"]
        })
    df.drop_duplicates(subset=["expenses_type"], keep="last", inplace=True)
    df.sort_values(by=["updated_at"], inplace=True)
    df = df.reset_index(drop=True)
    df["expenses_type_id"] = df.reset_index()['index'] + 1
    df = df.reindex(["expenses_type_id", "expenses_type", "updated_at"], axis=1)
    return df


def gen_expense_titles():
    df = pd.DataFrame(
        {
            "title": maintenance_expenses["title"],
            "expenses_type": maintenance_expenses["type"],
            "updated_at": maintenance_expenses["updated_at"],
        }
    )
    df.drop_duplicates(subset=["title"], keep="last", inplace=True)
    df = pd.merge(
        df,
        expense_types[["expenses_type_id", "expenses_type"]],
        how="left",
        on=["expenses_type"],
    )
    df.sort_values(by=["updated_at"], inplace=True)
    df = df.reset_index(drop=True)
    df["title_id"] = df.reset_index()['index'] + 1
    df = df.reindex(["title_id", "title", "expenses_type_id", "updated_at"], axis=1)
    return df


def _gen_tournament_game():
    games = prompt_games.loc[
        prompt_games["tournament"] == "TAK", ["name", "type", "category"]
    ]
    tournament_games = pd.merge(
        games,
        prompt_tournaments,
        left_on=["type", "category"],
        right_on=["type", "category"],
        suffixes=["_game", "_tournament"],
    )
    tournament_games = tournament_games[["name_game", "name_tournament"]]
    return tournament_games


def _gen_staff_handler():
    # 
    available_staff = prompt_staff_shifts.loc[
        (prompt_staff_shifts["weekday"] == config['event_info']["weekday"]) & (prompt_staff_shifts["hour"] >= config['event_info']["hour"])
    ]["staff_id"].unique()

    return np.random.choice(available_staff)


def count_matches(x):
    # na to jest gotowy wzór analityczny xDDDDDDDD
    total = 0
    while x >= 0:
        total += 2 ** x
        x -= 1
    return total


def gen_tournaments():
    dates = pd.DataFrame(
        prompt_dates.loc[prompt_dates["weekday"] == config['event_info']["weekday"], "date"].iloc[5::6] # added 5 because of the errors
    ).reset_index()["date"]
    tournament = _gen_tournament_game()
    tournament = tournament.sample(
        dates.shape[0], replace=False, ignore_index=True
    )  # nie może być ten sam wiersz
    tree_levels_number = randint.rvs(2, 4, size=dates.shape[0])
    matches = map(lambda x: count_matches(x), tree_levels_number)
    df = pd.DataFrame(
        {
            "name": tournament["name_tournament"],
            "game": tournament["name_game"],
            "start_time": dates + datetime.timedelta(hours=15),
            "matches": matches,
            "tree_levels": tree_levels_number,
            "fee": np.full((1, dates.shape[0]), 50)[0],
            "sign_up_deadline": dates
            - datetime.timedelta(days=3)
            + datetime.timedelta(hours=20),
            "staff_id": [_gen_staff_handler() for i in range(dates.shape[0])],
            "expenses": map(
                lambda x: round(x, 2), gamma.rvs(a=1.5, loc=200, scale=20, size=dates.shape[0])
            ),
            "updated_at": dates.apply(
                lambda x: x
                + datetime.timedelta(
                    hours=15,
                    minutes=random.randint(1, 5),
                    seconds=random.randint(1, 59),
                )
            ),
        }
    )
    df.sort_values(by=["updated_at"], inplace=True)
    df = df.reset_index(drop=True)
    df["tournament_id"] = df.reset_index()["index"] + 1
    df["invoice_id"] = df["tournament_id"]
    df = df.reindex(
        [
            "tournament_id",
            "name",
            "game",
            "start_time",
            "matches",
            "tree_levels",
            "fee",
            "sign_up_deadline",
            "staff_id",
            "expenses",
            "invoice_id",
            "updated_at",
        ],
        axis=1,
    )
    return df


def _sign_up_date_generator(deadline):
    deadline = pd.to_datetime(deadline)
    date = pd.to_datetime(
        prompt_dates["date"]
        .loc[
            (prompt_dates["date"] < deadline)
            & (prompt_dates["date"] > deadline - pd.DateOffset(days=14))
        ]
        .sample(1, ignore_index=True)
        .values[0]
    )
    date += pd.DateOffset(
        hours=random.choice(range(8, 20)),
        minutes=random.choice(range(0, 60)),
        seconds=random.choice(range(0, 60)),
    )
    return date


def gen_participations():
    # participants number
    players = pd.merge(
        tournaments[
            ["game", "tree_levels", "tournament_id", "fee", "sign_up_deadline"]
        ],
        prompt_games[["name", "participants_number"]],
        left_on=["game"],
        right_on=["name"],
        how="left",
    )
    players["participants_number"] = players["participants_number"].astype("int")
    players_number = players["participants_number"] * (2 ** players["tree_levels"])
    players["players"] = players_number

    # customer_id and place
    participants = map(
        lambda x: customers["customer_id"]
        .sample(x, replace=False, ignore_index=True)
        .values,
        players_number,
    )
    players["place"] = players["players"].apply(lambda x: [*range(1, x + 1)])
    players["customer_id"] = list(participants)

    df = players[["tournament_id", "customer_id", "fee", "sign_up_deadline", "place"]]
    df = df.explode(["customer_id", "place"], ignore_index=True)
    df["sign_up_date"] = df["sign_up_deadline"].apply(_sign_up_date_generator)
    df["updated_at"] = df["sign_up_date"].copy()
    df.sort_values(by=["updated_at"], inplace=True)
    df = df.reset_index(drop=True)
    df["particip_id"] = df.reset_index()['index'] + 1
    df["invoice_id"] = df.reset_index()['index'] + 1
    df = df.reindex(
        [
            "particip_id",
            "tournament_id",
            "customer_id",
            "place",
            "sign_up_date",
            "fee",
            "invoice_id",
            "updated_at",
        ],
        axis=1,
    )
    return df


v_repeat = np.vectorize(lambda date, volume: itertools.repeat(date, int(volume)))


def _gen_sell_inventory() -> pd.DataFrame:
    # evaluate a total game number
    total_games_n = np.round(
        prompt_dates["volume_sales"].sum()
        * (config['inventory_multiplier'] + np.random.exponential())
    )
    game_counts = list(map(round, prompt_games["weights"] * total_games_n))
    # repeat the games
    games_gener = itertools.chain(*v_repeat(prompt_games["name"], game_counts))
    games = np.array(list(games_gener))
    # add the basic details to the data frame
    s_inventory = pd.DataFrame(
        {
            "game": np.random.permutation(games),
            "destination": np.full(games.size, "S"),
            "active": np.full(games.size, True),
        }
    )
    # add the prices from the prompt table
    s_inventory = (
        pd.merge(
            s_inventory,
            prompt_games[["name", "purchase"]],
            left_on="game",
            right_on="name",
            how="inner",
        )
        .rename(columns={"purchase": "price"})
        .drop(columns="name")
    )
    # add the procurment prices as some fraction of the standard one
    s_inventory["purchase_payment"] = (s_inventory["price"] * config['bulk_ratio']).round(2)
    s_inventory = s_inventory.sample(n=s_inventory.shape[0]).reset_index(drop=True)
    # get the delivery dates
    delivery_count = config['avg_supply_yearly_rate'] * config['shop_lifetime_years']
    delivery_dates = prompt_dates["date"][
        :: int(prompt_dates.shape[0] / delivery_count)
    ].copy()
    delivery_dates += datetime.timedelta(hours=8)
    # the final grouping
    group_dates = np.repeat(
        delivery_dates.to_numpy(), np.ceil(total_games_n / delivery_count)
    )[: sum(game_counts)]
    s_inventory = s_inventory.reset_index(drop=True)
    s_inventory["delivery_date"] = group_dates
    return s_inventory


def _gen_rent_inventory() -> pd.DataFrame:
    total_games_n = config['rental_games_n']
    game_counts = list(map(round, prompt_games["weights"] * total_games_n))
    # repeat the games
    games_gener = itertools.chain(*v_repeat(prompt_games["name"], game_counts))
    games = np.array(list(games_gener))
    # active statuses
    active_status = np.full(games.size, True)
    active_status[:config['inactive_rental_games']] = False
    active_status = np.random.permutation(active_status)
    # add the basic details to the data frame
    r_inventory = pd.DataFrame(
        {
            "game": np.random.permutation(games),
            "destination": np.full(games.size, "R"),
            "active": active_status,
        }
    )
    # add the prices from the prompt table; temporarily prices are full
    r_inventory = (
        pd.merge(
            r_inventory,
            prompt_games[["name", "purchase"]],
            left_on="game",
            right_on="name",
            how="inner",
        )
        .rename(columns={"purchase": "price"})
        .drop(columns="name")
    )
    # add the procurment prices as some fraction of the standard one
    r_inventory["purchase_payment"] = (r_inventory["price"] * config['bulk_ratio']).round(2)
    # the delivery dates
    r_inventory["delivery_date"] = prompt_dates["date"].iloc[0] + datetime.timedelta(
        hours=8
    )
    r_inventory["price"] = (r_inventory["price"] * config['rental_price_ratio']).round(2)
    return r_inventory


def _gen_tournament_inventory() -> pd.DataFrame:
    # find the required game counts
    game_counts_tournaments = (
        tournaments[["game", "tree_levels"]]
        .groupby("game")
        .max()
        .apply(lambda x: 2 ** x)
    ).reset_index()
    # repeat the games
    games_gener = itertools.chain(
        *v_repeat(
            game_counts_tournaments["game"], game_counts_tournaments["tree_levels"]
        )
    )
    games = np.array(list(games_gener))

    # add the basic details to the data frame
    t_inventory = pd.DataFrame(
        {
            "game": np.random.permutation(games),
            "destination": np.full(games.size, "T"),
            "active": np.full(games.size, True),
        }
    )
    t_inventory = (
        pd.merge(
            t_inventory,
            prompt_games[["name", "purchase"]],
            left_on="game",
            right_on="name",
            how="inner",
        )
        .rename(columns={"purchase": "price"})
        .drop(columns="name")
    )
    # add the procurment prices as some fraction of the standard one
    t_inventory["purchase_payment"] = (t_inventory["price"] * config['bulk_ratio']).round(2)
    # the delivery dates
    t_inventory["delivery_date"] = prompt_dates["date"].iloc[0] + datetime.timedelta(
        hours=8
    )
    t_inventory["price"] = np.nan
    return t_inventory


def gen_inventory():
    # concat the parts
    inventory = pd.concat(
        [_gen_sell_inventory(), _gen_rent_inventory(), _gen_tournament_inventory()]
    )
    # add updated at value
    inventory["updated_at"] = inventory["delivery_date"]
    inventory = inventory.sort_values(by=["updated_at"])
    # index it
    inventory = inventory.reset_index(drop=True)
    inventory["inventory_id"] = inventory.reset_index()["index"] + 1
    # add invoice numbers
    uniq_dates = np.sort(np.unique(inventory["delivery_date"]))
    invoice_translator = dict(zip(uniq_dates, np.arange(uniq_dates.size) + 1))
    inventory["invoice_id"] = inventory["delivery_date"].map(invoice_translator)
    # export it
    inventory = inventory.reindex(
        [
            "inventory_id",
            "game",
            "destination",
            "price",
            "active",
            "purchase_payment",
            "invoice_id",
            "delivery_date",
            "updated_at",
        ],
        axis=1,
    )
    return inventory


def gen_game_prices():
    df = inventory.drop_duplicates(subset=["price"], keep="last", ignore_index=True)[
        ["price", "updated_at"]
    ]
    df.rename(columns={"price": "current_price"}, inplace=True)
    df = df.sort_values(["updated_at"])
    #set id
    df = df.reset_index(drop=True)
    df["price_id"] = df.reset_index()['index'] + 1
    df = df.reindex(["price_id", "current_price", "updated_at"], axis=1)
    return df


def gen_sales():
    v_repeat = np.vectorize(lambda date, volume: itertools.repeat(date, int(volume)))
    v_timedelta = np.vectorize(
        lambda h, m, s: datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
    )
    @np.vectorize
    def v_get_staff_id(timestamp):
        available_staff = prompt_staff_shifts[
            (prompt_staff_shifts["weekday"] == timestamp.weekday())
            & (prompt_staff_shifts["hour"] == timestamp.hour)
        ]["staff_id"]
        return np.random.choice(available_staff)


    # get a list of dates for each customer
    dates_gener = itertools.chain(
        *v_repeat(prompt_dates["date"], prompt_dates["volume_sales"])
    )
    dates = np.array(list(dates_gener))
    # add random time to the date
    hours = np.array(
        np.random.choice(prompt_hours["hour"], p=prompt_hours["prob"], size=dates.size)
    )
    minutes = np.random.randint(0, 59, size=dates.size)
    seconds = np.random.randint(0, 59, size=dates.size)
    timestamps = dates + v_timedelta(hours, minutes, seconds)
    timestamps.sort()
    # staff ids
    staff = v_get_staff_id(timestamps)
    # how many at once are bought
    pcs = np.random.choice(
        list(config['customer_pcs_probas'].keys()),
        p=list(config['customer_pcs_probas'].values()),
        size=dates.size,
    )
    # make a data frame and add invoices as ix
    sales = pd.DataFrame({"date": timestamps, "staff_id": staff, "pcs": pcs})
    sales = sales.reset_index().rename(columns={"index": "invoice"})
    # additional rows when the pieces number is larger than one
    for pcs_n in config['customer_pcs_probas'].keys():
        selection = sales[sales["pcs"] == pcs_n]
        sales = pd.concat([sales] + [selection] * (pcs_n - 1))
    sales = sales.reset_index(drop=True).drop("pcs", axis="columns")
    # pre-filter the inventory table
    global inventory # DELETE THAT!!!!!!!!!!!
    sales_inventory = inventory[inventory["destination"] == "S"]
    sales = pd.concat(
        [
            sales,
            sales_inventory.iloc[: sales.shape[0]].reset_index()[
                ["inventory_id", "price", "delivery_date"]
            ],
        ],
        axis=1,
    )
    sales = sales[~(sales["date"] < sales["delivery_date"])]
    sales = sales.drop(columns=["delivery_date"])
    inventory.loc[sales["inventory_id"], "active"] = False
   
    # other details
    sales["updated_at"] = sales["date"]
    sales["return_oper"] = False
    # export
    sales.sort_values("updated_at", inplace=True)
    # fix
    sales = sales.reset_index(drop=True)
    sales["sale_id"] = sales.reset_index()["index"] + 1
    sales = sales.reindex(
        [
            "sale_id",
            "inventory_id",
            "staff_id",
            "price",
            "date",
            "invoice",
            "return_oper",
            "updated_at",
        ],
        axis=1,
    )
    return sales


def gen_rental():
    rental_date = []
    return_dates = []
    customer = []
    price = []
    staff = []
    inventory_ids = []
    rate = []
    update = []
    penalty = []
    invoice = []
    penalty_invoice = []

    @np.vectorize
    def v_get_staff_id(timestamp):
        available_staff = prompt_staff_shifts[
            (prompt_staff_shifts["weekday"] == timestamp.weekday())
            & (prompt_staff_shifts["hour"] == timestamp.hour)
        ]["staff_id"]
        return np.random.choice(available_staff)
    
    def v_proper_return_date(timestamp, date):
        if (timestamp.day == date.day) & (timestamp.hour < date.hour):
            return timestamp.replace(hour=config['shop_open_hours']['from'])
        elif timestamp.hour < config['shop_open_hours']['from']:
            return timestamp.replace(hour=config['shop_open_hours']['from'])
        elif timestamp.hour >= config['shop_open_hours']['to']:
            return timestamp.replace(hour=config['shop_open_hours']['to']-1)
        else:
            return timestamp

    for j in range(prompt_dates["date"].shape[0]):
        for i in range(int(prompt_dates["volume_rental"][j])):
            hour = random.choices(
                prompt_hours["hour"], weights=prompt_hours["prob"], k=1
            )[0]
            date = prompt_dates["date"][j] + datetime.timedelta(
                hours=int(hour),
                minutes=random.randint(0, 59),
                seconds=random.randint(0, 59),
            )
            game = np.random.choice(prompt_games["name"], p=prompt_games["weights"])
            staff_id = v_get_staff_id(date)
            holding_time = gamma.rvs(scale=3, a=2)
            return_date_t = prompt_dates["date"][j] + datetime.timedelta(
                days=int(holding_time),
                hours=round(abs(holding_time) % 1 * 24, 2),
                minutes=random.randint(0, 59),
                seconds=random.randint(0, 59),
            )
            return_date = v_proper_return_date(return_date_t, date)

            mask = (
                (inventory["game"] == game)
                & (inventory["delivery_date"] < date)
                & (inventory["destination"] == "R")
                & (inventory["active"] == True)
            )
            if mask.any():
                customer.append(int(random.choice(customers["customer_id"])))
                price.append(inventory["price"].loc[mask.idxmax()])
                rental_date.append(date)
                staff.append(staff_id)
                game_inventory_id = inventory["inventory_id"].loc[mask.idxmax()]
                inventory_ids.append(game_inventory_id)
                invoice.append(f"{i}{j}")
                if return_date > datetime.datetime.today():
                    return_dates.append(np.nan)
                    update.append(date)
                    inventory.loc[mask.idxmax(), "active"] = False
                    penalty.append(np.nan)
                    penalty_invoice.append(np.nan)
                else:
                    return_dates.append(return_date)
                    update.append(return_date)
                    delta = return_date - date
                    if delta.days > config['rental_allowed_days']:
                        penalty.append(
                            (
                                (
                                    inventory[
                                        inventory["inventory_id"] == game_inventory_id
                                    ]["price"]
                                )
                                * config['penalty_ratio']
                            )
                            .to_numpy()[0]
                            .round(2)
                            * delta.days
                        )
                        penalty_invoice.append(f"P{i}_{j}")
                    else:
                        penalty.append(np.nan)
                        penalty_invoice.append(np.nan)
                rate.append(random.randint(1, 10))

    df = pd.DataFrame(
        {
            "inventory_id": inventory_ids,
            "customer_id": customer,
            "rental_date": rental_date,
            "return_date": return_dates,
            "staff_id": staff,
            "price": price,
            "invoice": invoice,
            "penalty_payment": penalty,
            "penalty_invoice": penalty_invoice,
            "rate": rate,
            "updated_at": update,
        }
    )
    df.sort_values("updated_at", inplace=True)
    df = df.reset_index(drop=True)
    df["rental_id"] = df.reset_index()['index'] + 1
    df = df.reindex(
        [
            "rental_id",
            "inventory_id",
            "customer_id",
            "rental_date",
            "return_date",
            "staff_id",
            "price",
            "invoice",
            "penalty_payment",
            "penalty_invoice",
            "rate",
            "updated_at",
        ],
        axis=1,
    )
    return df


def gen_games():
    update = inventory[["game", "delivery_date"]].groupby(["game"]).min()
    games = pd.merge(prompt_games, update, left_on=["name"], right_on=["game"])
    games = games[["name", "description", "category", "type", "tournament", "delivery_date"]]
    games.rename(
        columns={
            "name": "title",
            "description": "description",
            "category": "category",
            "type": "type",
            "tournament": "competitivity",
            "delivery_date": "updated_at",
        },
        inplace=True,
    )
    games["competitivity"] = games["competitivity"].apply(
        lambda x: True if x == "TAK" else False
    )
    games.sort_values(["updated_at"], inplace=True)
    games = games.reset_index(drop=True)
    games["game_id"] = games.reset_index()['index'] + 1
    games = games.reindex(
        [
            "game_id",
            "title",
            "description",
            "category",
            "type",
            "competitivity",
            "updated_at",
        ],
        axis=1,
    )
    return games


def gen_game_categories():
    df = games[["category", "updated_at"]].drop_duplicates("category", keep="first")
    df.rename(columns={"category": "game_category"}, inplace=True)
    df = df.reset_index(drop=True)
    df["category_id"] = df.reset_index()['index'] + 1
    df = df.reindex(["category_id", "game_category", "updated_at"], axis=1)
    return df


def gen_game_types():
    df = games[["type", "updated_at"]].drop_duplicates("type", keep="first")
    df.rename(columns={"type": "game_type"}, inplace=True)
    df = df.reset_index(drop=True)
    df["type_id"] = df.reset_index()['index'] + 1
    df = df.reindex(["type_id", "game_type", "updated_at"], axis=1)
    return df


def _customers_updated():
    global customers
    active_customers = pd.concat([rental[['customer_id', 'updated_at']], participations[['customer_id', 'updated_at']]], ignore_index=True)
    customers = pd.merge(customers, active_customers.groupby(by='customer_id').min(), on='customer_id')
    customers.sort_values('updated_at', inplace=True)
    customers.reset_index(drop=True, inplace=True)
    return customers


def gen_real_customers():
    global customers
    customers = _customers_updated()
    customers["gender"] = np.random.choice(["M", "F"], size=customers.shape[0])
    customers["phone"] = _gen_one_phone(np.zeros(customers.shape[0]))
    customers["city"] = np.random.choice(prompt_cities["city"], p=prompt_cities["prob"], size=customers.shape[0])
    customers["first_name"] = _gen_one_name(customers["gender"], "first")
    customers["last_name"] = _gen_one_name(customers["gender"], "last")
    customers["email"] = _gen_email(customers["first_name"], customers["last_name"])
    customers['previous_customer_id'] = customers['customer_id']
    customers['customer_id'] = customers.reset_index()['index'] +1
    customers = customers.reindex(columns=[
        'customer_id',
        'previous_customer_id',
        'first_name',
        'last_name',
        'phone',
        'email',
        'city',
        'updated_at'
    ])
    return customers


def gen_cities():
    merged_df = pd.concat([customers, staff], ignore_index=True)
    merged_df.sort_values(["updated_at"], inplace=True)
    merged_df.drop_duplicates(subset=["city"], keep="first", inplace=True)
    city = pd.DataFrame(
        {
            "city": merged_df["city"],
            "updated_at": merged_df["updated_at"],
        }
    )
    city = city.reset_index(drop=True)
    city['city_id'] = city.reset_index()['index'] + 1
    city = city.reindex([
        'city_id',
        'city',
        'updated_at'
    ], axis=1)
    return city


def _gen_working_payments():
    # inventory
    df_inv = inventory[["purchase_payment", "invoice_id", "updated_at"]].rename(
        columns={
            "purchase_payment": "amount",
            "invoice_id": "invoice",
            "updated_at": "date",
        }
    )
    df_inv["invoice"] = "I" + df_inv["invoice"].astype(str)
    df_inv["amount"] = -df_inv["amount"]
    # sales
    df_sales = sales[["price", "invoice", "date"]].rename(
        columns={"price": "amount"}
    )
    df_sales["invoice"] = "S" + df_sales["invoice"].astype(str)
    # rent
    df_rent = rental[["price", "invoice", "rental_date"]].rename(
        columns={"price": "amount", "rental_date": "date"}
    )
    df_rent["invoice"] = "R" + df_rent["invoice"].astype(str)
    # rent penalty
    df_rent_penalty = (
        rental[["penalty_payment", "penalty_invoice", "return_date"]]
        .dropna()
        .rename(
            columns={
                "penalty_payment": "amount",
                "penalty_invoice": "invoice",
                "return_date": "date",
            }
        )
    )
    df_rent_penalty["invoice"] = "R" + df_rent_penalty["invoice"].astype(str)
    # expenses
    df_exp = maintenance_expenses[["amount", "invoice_id", "updated_at"]].rename(
        columns={"updated_at": "date", "invoice_id": "invoice"}
    )
    df_exp["invoice"] = "ME" + df_exp["invoice"].astype(str)
    df_exp["amount"] = -df_exp["amount"]
    # tour
    df_tour = tournaments[["expenses", "invoice_id", "updated_at"]].rename(
        columns={"expenses": "amount", "invoice_id": "invoice", "updated_at": "date"}
    )
    df_tour["invoice"] = "T" + df_tour["invoice"].astype(str)
    df_tour["amount"] = -df_tour["amount"]
    # participations
    df_part = participations[["fee", "invoice_id", "updated_at"]].rename(
        columns={"fee": "amount", "invoice_id": "invoice", "updated_at": "date"}
    )
    df_part["invoice"] = "P" + df_part["invoice"].astype(str)
    df = pd.concat(
        [df_inv, df_sales, df_rent, df_rent_penalty, df_exp, df_tour, df_part],
        ignore_index=True,
    )
    return df


def gen_invoices():
    df = working_payments[["invoice", "date"]]
    df = df.drop_duplicates("invoice")
    df["updated_at"] = df.loc[:, "date"]
    df = df.reset_index(drop=True)
    df["invoice_id"] = df.reset_index()['index'] + 1
    df = df.reindex(["invoice_id", "invoice", "date", "updated_at"], axis=1)
    return df


def gen_payments():
    df = pd.merge(working_payments, invoices, on="invoice")[
        ["amount", "invoice_id", "updated_at"]
    ]
    df = df.reset_index(drop=True)
    df["payment_id"] = df.reset_index()['index'] + 1
    df = df.reindex(["payment_id", "amount", "invoice_id", "updated_at"], axis=1)
    return df


def customers_cleaning():
    global customers
    customers["city"] = customers["city"].map(
        dict(zip(city["city"], city["city_id"]))
    )
    customers = customers.rename(columns={'city': 'city_id'})
    customers.drop(columns=['previous_customer_id'], inplace=True)


def participations_cleaning():
    global participations
    participations.rename(columns={'fee': 'fee_payment_id'}, inplace=True)
    participations['customer_id'] = participations['customer_id'].map(
        dict(zip(customers['previous_customer_id'], customers['customer_id']))
    )
    participations['fee_payment_id'] = participations['updated_at'].map(
        dict(zip(payments['updated_at'], payments['payment_id']))
    )
    participations.drop(columns=['invoice_id'], inplace=True)


def tournaments_cleaning():
    global tournaments
    tournaments["game"] = tournaments["game"].map(
        dict(zip(games["title"], games["game_id"]))
    )
    tournaments = tournaments.rename(
    columns={"game": "game_id", "expenses": "expenses_payments_id"})
    tournaments['expenses_payments_id'] = tournaments['updated_at'].map(
        dict(zip(payments['updated_at'], payments['payment_id']))
    )
    tournaments.drop(columns=["tree_levels", "invoice_id"], inplace=True)


def rental_cleaning():
    global rental
    rental['customer_id'] = rental['customer_id'].map(
        dict(zip(customers['previous_customer_id'], customers['customer_id']))
    )
    rental = rental.rename(columns={"penalty_payment": "penalty_payment_id", "price": "payment_id"})
    rental.drop(columns=['invoice', 'penalty_invoice'], inplace=True)


def inventory_cleaning():
    global inventory
    inventory["game"] = inventory["game"].map(
        dict(zip(games["title"], games["game_id"]))
    )
    inventory = inventory.rename(
        columns={
            "game": "game_id",
            "price": "price_id",
            "purchase_payment": "purchase_payment_id",
            }
        )
    inventory['purchase_payment_id'] = inventory['updated_at'].map(
        dict(zip(payments['updated_at'], payments['payment_id']))
    )
    inventory['price_id'] = inventory['price_id'].map(
        dict(zip(game_prices['current_price'], game_prices['price_id']))
    )
    inventory.drop(columns=['invoice_id'], inplace=True)


def staff_cleaning():
    global staff
    temp = dict(zip(city["city"], city["city_id"]))
    staff["city"] = staff["city"].map(temp)
    staff.rename(columns={"city": "city_id"}, inplace=True)


def invoices_cleaning():
    global invoices
    invoices.drop(columns=["invoice"], inplace=True)
        

def maintenance_expenses_cleaning():
    global maintenance_expenses
    maintenance_expenses["title"] = maintenance_expenses["title"].map(
        dict(zip(expense_titles["title"], expense_titles["title_id"]))
    )
    maintenance_expenses = maintenance_expenses.drop(
        columns=["invoice_id", "amount", "type"]
    )
    maintenance_expenses = maintenance_expenses.rename(columns={"title": "title_id"})
    maintenance_expenses['payment_id'] = maintenance_expenses['updated_at'].map(
        dict(zip(payments['updated_at'], payments['payment_id']))
    )
    maintenance_expenses = maintenance_expenses.reindex([
        'spend_id',
        'title_id',
        'payment_id',
        'date',
        'updated_at'], axis=1)


def sales_cleaning():
    global sales
    sales = sales.rename(columns={'price': 'payment_id'})
    sales['payment_id'] = sales['date'].map(
        dict(zip(payments['updated_at'], payments['payment_id']))
    )
    sales.drop(columns=['invoice'], inplace=True)


def games_cleaning():
    global games
    games["category"] = games["category"].map(
        dict(zip(game_categories["game_category"], game_categories["category_id"]))
    )
    games["type"] = games["type"].map(
        dict(zip(game_types["game_type"], game_types["type_id"]))
    )
    games = games.rename(columns={"category": "category_id", "type": "type_id"})

In [7]:
prompt_dates = gen_prompt_dates()
prompt_hours = gen_prompt_hours()
prepar_prompt_games()
staff = gen_staff()
prompt_staff_shifts = gen_prompt_staff_shifts()
relationships = gen_realtionships()
partners = gen_partners()
customers = pd.DataFrame({"customer_id": [*range(1, config['customers_number'] + 1)]})
maintenance_expenses = gen_maintenance_expenses()
expense_types = gen_expense_types()
expense_titles = gen_expense_titles()
tournaments = gen_tournaments()
participations = gen_participations()
inventory = gen_inventory()
game_prices = gen_game_prices()
sales = gen_sales()
rental = gen_rental()
games = gen_games()
game_categories = gen_game_categories()
game_types = gen_game_types()
customers = gen_real_customers()
city = gen_cities()
working_payments = _gen_working_payments()
invoices = gen_invoices()
payments = gen_payments()
participations_cleaning()
tournaments_cleaning()
rental_cleaning()
customers_cleaning()
inventory_cleaning()
staff_cleaning()
invoices_cleaning()
maintenance_expenses_cleaning()
sales_cleaning()
games_cleaning()

### Sprawdzenie

In [8]:
city.head()

,city_id,city,updated_at
0,1,Wrocław,2021-07-01 08:17:19
1,2,Polkowice,2021-07-04 08:10:20
2,3,Piława Górna,2021-07-04 08:11:43
3,4,Bystrzyca Kłodzka,2021-07-04 08:30:08
4,5,Legnica,2021-07-04 08:34:43


In [9]:
customers.head()

,customer_id,first_name,last_name,phone,email,city_id,updated_at
0,1,MACIEJ,RZĄCA,500122839,maciej.rzaca@outlook.com,1,2021-07-02 19:38:59
1,2,DANUTA,GRYGIER,720399359,danuta.grygier@go2.pl,2,2021-07-04 08:10:20
2,3,LENA,BIELECKA,669035482,lena.bielecka@outlook.com,3,2021-07-04 08:11:43
3,4,ANDRZEJ,TARASZKIEWICZ,511183673,andrzej.taraszkiewicz@onet.pl,4,2021-07-04 08:30:08
4,5,LEON,KOWALCZUK,787584099,leon.kowalczuk@interia.pl,5,2021-07-04 08:34:43


In [10]:
participations.head()

,particip_id,tournament_id,customer_id,place,sign_up_date,fee_payment_id,updated_at
0,1,1,45,14,2021-07-21 11:13:10,66438,2021-07-21 11:13:10
1,2,1,76,8,2021-07-21 11:43:29,66439,2021-07-21 11:43:29
2,3,1,80,2,2021-07-21 15:35:20,66440,2021-07-21 15:35:20
3,4,1,81,48,2021-07-21 17:59:52,66441,2021-07-21 17:59:52
4,5,1,82,28,2021-07-21 18:13:18,66442,2021-07-21 18:13:18


In [11]:
tournaments.head()

,tournament_id,name,game_id,start_time,matches,fee,sign_up_deadline,staff_id,expenses_payments_id,updated_at
0,1,Potyczka Prawdziwych Kolosów,46,2021-08-06 15:00:00,7,50,2021-08-03 20:00:00,4,66421,2021-08-06 15:01:27
1,2,Krucjata Karcianych Mistrzów,45,2021-09-17 15:00:00,7,50,2021-09-14 20:00:00,4,66422,2021-09-17 15:02:04
2,3,Wrocławski Konwent Planszówkowy,12,2021-10-29 15:00:00,7,50,2021-10-26 20:00:00,6,66423,2021-10-29 15:04:41
3,4,Krucjata Karcianych Mistrzów,28,2021-12-10 15:00:00,7,50,2021-12-07 20:00:00,3,66424,2021-12-10 15:03:56
4,5,Cytadela Strategii,31,2022-01-21 15:00:00,15,50,2022-01-18 20:00:00,6,66425,2022-01-21 15:02:54


In [12]:
rental

,rental_id,inventory_id,customer_id,rental_date,return_date,staff_id,payment_id,penalty_payment_id,rate,updated_at
0,1,1938,1,2021-07-02 16:23:28,2021-07-02 19:38:59,4,12.50,NaN,10,2021-07-02 19:38:59
1,2,2058,2,2021-07-01 19:04:49,2021-07-04 08:10:20,3,29.60,NaN,9,2021-07-04 08:10:20
2,3,2047,3,2021-07-02 16:27:37,2021-07-04 08:11:43,4,8.98,NaN,5,2021-07-04 08:11:43
3,4,1938,4,2021-07-01 18:34:34,2021-07-04 08:30:08,4,12.50,NaN,4,2021-07-04 08:30:08
4,5,1931,5,2021-07-02 15:23:19,2021-07-04 08:34:43,6,12.00,NaN,3,2021-07-04 08:34:43
...,...,...,...,...,...,...,...,...,...,...
4782,4783,1946,897,2023-06-30 14:20:15,NaT,4,16.49,NaN,6,2023-06-30 14:20:15
4783,4784,2051,287,2023-06-30 15:16:32,NaT,6,6.18,NaN,10,2023-06-30 15:16:32
4784,4785,2059,620,2023-06-30 15:44:27,NaT,4,29.60,NaN,8,2023-06-30 15:44:27
4785,4786,1947,42,2023-06-30 16:12:24,NaT,6,24.38,NaN,7,2023-06-30 16:12:24


In [13]:
inventory.head()

,inventory_id,game_id,destination,price_id,active,purchase_payment_id,delivery_date,updated_at
0,1,3,S,49,True,6083,2021-07-01 08:00:00,2021-07-01 08:00:00
1,2,35,S,80,False,6083,2021-07-01 08:00:00,2021-07-01 08:00:00
2,3,38,S,56,False,6083,2021-07-01 08:00:00,2021-07-01 08:00:00
3,4,40,S,95,False,6083,2021-07-01 08:00:00,2021-07-01 08:00:00
4,5,26,S,81,False,6083,2021-07-01 08:00:00,2021-07-01 08:00:00


In [14]:
staff.head()

,staff_id,first_name,last_name,phone,email,city_id,current_salary,is_manager,gender,from_date,to_date,updated_at
0,1,ROBERT,GAWRYCH,736241466,robert.gawrych@o2.pl,1,3637.60,False,M,2021-07-01,NaN,2021-07-01 08:17:19
1,2,MARIA,WRÓBEL,781228316,maria.wrobel@o2.pl,1,3544.15,False,F,2021-07-01,NaN,2021-07-01 09:42:12
2,3,GRZEGORZ,KUBICKI,501074693,grzegorz.kubicki@vp.pl,1,3584.05,False,M,2021-07-01,NaN,2021-07-01 14:02:31
3,4,KAZIMIERZ,BOLESTA,604219770,kazimierz.bolesta@tlen.pl,1,3551.31,False,M,2021-07-01,NaN,2021-07-01 17:07:37
4,5,NATALIIA,KOSIŃSKA,516820162,nataliia.kosinska@gazeta.pl,1,NaN,False,F,2021-07-01,2022-01-10,2022-01-10 09:18:39


In [15]:
relationships.head()

,relationship_id,staff_id,partner_id,dates_number,updated_at
0,1,4,1,5,2021-07-09 14:50:21
1,2,2,2,8,2021-12-07 08:01:39
2,3,4,3,4,2022-03-12 16:35:51
3,4,2,4,2,2022-08-22 14:50:14
4,5,1,5,7,2022-11-16 17:47:10


In [16]:
partners.head()

,partner_id,name,gender,updated_at
0,1,EWA,F,2021-07-09 14:50:21
1,2,JÓZEF,M,2021-12-07 08:01:39
2,3,DANUTA,F,2022-03-12 16:35:51
3,4,ARKADIUSZ,M,2022-08-22 14:50:14
4,5,BARBARA,F,2022-11-16 17:47:10


In [17]:
payments.head()

,payment_id,amount,invoice_id,updated_at
0,1,-11.90,1,2021-07-01 08:00:00
1,2,-93.39,1,2021-07-01 08:00:00
2,3,-18.55,1,2021-07-01 08:00:00
3,4,-21.32,1,2021-07-01 08:00:00
4,5,-14.09,1,2021-07-01 08:00:00


In [18]:
invoices.head()

,invoice_id,date,updated_at
0,1,2021-07-01 08:00:00,2021-07-01 08:00:00
1,2,2021-10-27 08:00:00,2021-10-27 08:00:00
2,3,2022-02-28 08:00:00,2022-02-28 08:00:00
3,4,2022-06-29 08:00:00,2022-06-29 08:00:00
4,5,2022-10-26 08:00:00,2022-10-26 08:00:00


In [19]:
maintenance_expenses.head()

,spend_id,title_id,payment_id,date,updated_at
0,1,1,66230,2021-07-05,2021-07-05 08:09:23
1,2,16,66231,2021-07-05,2021-07-05 09:26:11
2,3,17,66232,2021-07-05,2021-07-05 10:30:11
3,4,22,66233,2021-07-05,2021-07-05 11:09:20
4,5,21,66234,2021-07-05,2021-07-05 16:26:54


In [20]:
expense_titles.head()

,title_id,title,expenses_type_id,updated_at
0,1,PENSJA LIPIEC NATALIIA KOSIŃSKA,3,2021-07-05 08:09:23
1,2,PENSJA SIERPIEŃ NATALIIA KOSIŃSKA,3,2021-08-05 13:20:11
2,3,PENSJA WRZESIEŃ NATALIIA KOSIŃSKA,3,2021-09-06 15:25:41
3,4,PENSJA PAŹDZIERNIK NATALIIA KOSIŃSKA,3,2021-10-05 14:04:43
4,5,PENSJA LISTOPAD NATALIIA KOSIŃSKA,3,2021-11-05 10:26:44


In [21]:
expense_types.head()

,expenses_type_id,expenses_type,updated_at
0,1,MEDIA,2023-06-05 11:47:28
1,2,CZYNSZ,2023-06-05 12:18:27
2,3,PENSJA,2023-06-05 18:29:28


In [22]:
sales.head()

,sale_id,inventory_id,staff_id,payment_id,date,return_oper,updated_at
0,1,1,2,35778,2021-07-01 10:20:10,False,2021-07-01 10:20:10
1,2,2,1,35779,2021-07-01 10:41:17,False,2021-07-01 10:41:17
2,3,3,2,35780,2021-07-01 11:01:42,False,2021-07-01 11:01:42
3,4,4,1,35781,2021-07-01 11:07:20,False,2021-07-01 11:07:20
4,5,5,2,35782,2021-07-01 11:13:52,False,2021-07-01 11:13:52


In [23]:
games.head()

,game_id,title,description,category_id,type_id,competitivity,updated_at
0,1,Jamniki,"Podczas gry Twój jamnik będzie rósł, musisz wi...",1,1,False,2021-07-01 08:00:00
1,2,"Karty dżentelmenów, Epizod trzeci",Karty Dżentelmenów: Epizod Trzeci to towarzysk...,2,2,False,2021-07-01 08:00:00
2,3,Super Farmer,Super Farmer to kultowa gra rodzinna o strateg...,1,3,True,2021-07-01 08:00:00
3,4,Pędzące Żółwie,Celem gry “Pędzące żółwie” jest doprowadzenie ...,1,1,False,2021-07-01 08:00:00
4,5,"Buła, Pizza, Kot, Ser, Koza","Buła, Pizza, Kot, Ser, Koza - gra równie szalo...",2,2,False,2021-07-01 08:00:00


In [24]:
game_categories.head()

,category_id,game_category,updated_at
0,1,familijna,2021-07-01 08:00:00
1,2,towarzyska,2021-07-01 08:00:00
2,3,narracyjna,2021-07-01 08:00:00
3,4,strategiczna,2021-07-01 08:00:00
4,5,logiczna,2021-07-01 08:00:00


In [25]:
game_types.head()

,type_id,game_type,updated_at
0,1,planszowa,2021-07-01 08:00:00
1,2,karciana,2021-07-01 08:00:00
2,3,inna,2021-07-01 08:00:00


In [26]:
game_prices.head()

,price_id,current_price,updated_at
0,1,9.18,2021-07-01 08:00:00
1,2,23.98,2021-07-01 08:00:00
2,3,53.37,2021-07-01 08:00:00
3,4,57.80,2021-07-01 08:00:00
4,5,10.81,2021-07-01 08:00:00
